In [133]:
import numpy as np 
import pandas as pd

In [134]:
summary = pd.read_csv("message_summary.csv",encoding='iso-8859-1')
summary.head()

,headline,text
0,buy best budget phone,I need to buy best budget phone
1,buy best budget phone,I need to buy best budget phone
2,buy best budget phone,I need to buy best budget phone
3,buy best budget phone,I need to buy best budget phone
4,buy best budget phone,I need to buy best budget phone


In [135]:
raw = pd.read_csv('message_summary.csv', encoding='iso-8859-1')
raw.head()

,headline,text
0,buy best budget phone,I need to buy best budget phone
1,buy best budget phone,I need to buy best budget phone
2,buy best budget phone,I need to buy best budget phone
3,buy best budget phone,I need to buy best budget phone
4,buy best budget phone,I need to buy best budget phone


In [136]:
pre1 =  raw.iloc[:,0:2].copy()
pre1['headline + text'] = pre1['headline'].str.cat(pre1['text'], sep =" ")

In [137]:
pre1.head()

,headline,text,headline + text
0,buy best budget phone,I need to buy best budget phone,buy best budget phone I need to buy best budget phone
1,buy best budget phone,I need to buy best budget phone,buy best budget phone I need to buy best budget phone
2,buy best budget phone,I need to buy best budget phone,buy best budget phone I need to buy best budget phone
3,buy best budget phone,I need to buy best budget phone,buy best budget phone I need to buy best budget phone
4,buy best budget phone,I need to buy best budget phone,buy best budget phone I need to buy best budget phone


In [138]:
pre2 = summary.iloc[:,0:6].copy()
pre2['head line+ text'] = pre2['headline'].str.cat(pre2['text'],sep= " ")

In [139]:
pre = pd.DataFrame()
pre['text'] = pd.concat([pre1['text'], pre2['text']], ignore_index=True)
pre['summary'] = pd.concat([pre1['headline'],pre2['headline']],ignore_index = True)

In [140]:
pre['text'][:4]

0    I need to buy best budget phone
1    I need to buy best budget phone
2    I need to buy best budget phone
3    I need to buy best budget phone
Name: text, dtype: object

In [141]:
import re 
def text_strip(column):

  for row in column:    
    row=re.sub("(\\t)", ' ', str(row)).lower() 
    row=re.sub("(\\r)", ' ', str(row)).lower() 
    row=re.sub("(\\n)", ' ', str(row)).lower()
    
    row=re.sub("(__+)", ' ', str(row)).lower()   
    row=re.sub("(--+)", ' ', str(row)).lower()   
    row=re.sub("(~~+)", ' ', str(row)).lower()   
    row=re.sub("(\+\++)", ' ', str(row)).lower()   
    row=re.sub("(\.\.+)", ' ', str(row)).lower()  
    
    row=re.sub(r"[<>()|&©ø\[\]\'\",;?~*!]", ' ', str(row)).lower() 
    
    row=re.sub("(mailto:)", ' ', str(row)).lower() 
    row=re.sub(r"(\\x9\d)", ' ', str(row)).lower() 
    row=re.sub("([iI][nN][cC]\d+)", 'INC_NUM', str(row)).lower() 
    row=re.sub("([cC][mM]\d+)|([cC][hH][gG]\d+)", 'CM_NUM', str(row)).lower() 
    
    
    row=re.sub("(\.\s+)", ' ', str(row)).lower() 
    row=re.sub("(\-\s+)", ' ', str(row)).lower() 
    row=re.sub("(\:\s+)", ' ', str(row)).lower() 
    
    row=re.sub("(\s+.\s+)", ' ', str(row)).lower() 

    # to replace an url
    try:
        url = re.search(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', str(row))
        repl_url = url.group(3)
        row = re.sub(r'((https*:\/*)([^\/\s]+))(.[^\s]+)',repl_url, str(row))
    except:
        pass
    

    
    row = re.sub("(\s+)",' ',str(row)).lower() #remove multiple spaces
    
    row=re.sub("(\s+.\s+)", ' ', str(row)).lower()

    
    
    yield row

In [142]:
brief_cleaning1 = text_strip(pre['text'])
brief_cleaning2 = text_strip(pre['summary'])

In [143]:
print(brief_cleaning1)

<generator object text_strip at 0x7f0b038147d0>


In [144]:
from time import time
import spacy
nlp = spacy.load('en', disable=['ner', 'parser']) 

t = time()
#spacy has the pipe method to increase the speed of prepocessing
#we need it because we are removing the urls and all the stuff from nearly 100k rows
text = [str(doc) for doc in nlp.pipe(brief_cleaning1, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 0.0 mins


In [145]:
t = time()

summary = ['SOS '+ str(doc) + ' EOS' for doc in nlp.pipe(brief_cleaning2, batch_size=5000, n_threads=-1)]
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 0.0 mins


In [146]:
text[0]

'i need to buy best budget phone'

In [147]:
summary[0]

'SOS buy best budget phone EOS'

In [148]:
pre['cleaned_text'] = pd.Series(text)
pre['cleaned_summary'] = pd.Series(summary)

In [149]:
text_count = []
summary_count = []

for sent in pre['cleaned_text']:
    text_count.append(len(sent.split()))
for sent in pre['cleaned_summary']:
    summary_count.append(len(sent.split()))

graph_df= pd.DataFrame()
graph_df['text']=text_count
graph_df['summary']=summary_count

In [150]:
#Check how much % of summary have 0-15 words
cnt=0
for i in pre['cleaned_summary']:
    if(len(i.split())<=15):
        cnt=cnt+1
print(cnt/len(pre['cleaned_summary']))

1.0


In [151]:
#Check how much % of text have 0-60 words
cnt=0
for i in pre['cleaned_text']:
    if(len(i.split())<=60):
        cnt=cnt+1
print(cnt/len(pre['cleaned_text']))

1.0


In [152]:
max_text_len=65
max_summary_len=15

cleaned_text =np.array(pre['cleaned_text'])
cleaned_summary=np.array(pre['cleaned_summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])
        
post_pre=pd.DataFrame({'text':short_text,'summary':short_summary})

In [153]:
post_pre.head(2)

,text,summary
0,i need to buy best budget phone,SOS buy best budget phone EOS
1,i need to buy best budget phone,SOS buy best budget phone EOS


In [154]:
post_pre['summary'] = post_pre['summary'].apply(lambda x : 'sostok '+ x + ' eostok')

In [155]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(np.array(post_pre['text']),np.array(post_pre['summary']),test_size=0.1,random_state=0,shuffle=True)

In [156]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

In [157]:
thresh_hold=4

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh_hold):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 0.0
Total Coverage of rare words: 0.0


In [158]:
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences (i.e one-hot encodeing all the words)
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

#size of vocabulary ( +1 for padding token)
x_voc   =  x_tokenizer.num_words + 1

print("Size of vocabulary in X = {}".format(x_voc))

Size of vocabulary in X = 8


In [159]:
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

In [160]:
thresh_hold=6

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh_hold):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 0.0
Total Coverage of rare words: 0.0


In [161]:
y_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
y_tokenizer.fit_on_texts(list(y_tr))

#convert text sequences into integer sequences (i.e one hot encode the text in Y)
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen=max_summary_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_summary_len, padding='post')

#size of vocabulary
y_voc  =   y_tokenizer.num_words +1
print("Size of vocabulary in Y = {}".format(y_voc))

Size of vocabulary in Y = 9


In [162]:
ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)

In [163]:
ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

In [164]:
from keras import backend as K 
import gensim
from numpy import *
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

print("Size of vocabulary from the w2v model = {}".format(x_voc))

K.clear_session()

latent_dim = 300
embedding_dim=200

# Encoder
encoder_inputs = Input(shape=(max_text_len,))

#embedding layer
enc_emb =  Embedding(x_voc, embedding_dim,trainable=True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

Size of vocabulary from the w2v model = 8
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 65)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 65, 200)      1600        input_11[0][0]                   
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 65, 300), (N 601200      embedding[0][0]                  
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, None)]       0                                            
____________________________________________________

In [165]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)

In [166]:
history=model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=50,callbacks=[es],batch_size=128, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

Epoch 1/50
1/1 [==============================] - 13s 13s/step - loss: 2.1918 - val_loss: 1.9339
Epoch 2/50
1/1 [==============================] - 3s 3s/step - loss: 1.9358 - val_loss: 1.2860
Epoch 3/50
1/1 [==============================] - 3s 3s/step - loss: 1.2969 - val_loss: 3.2405
Epoch 4/50
1/1 [==============================] - 3s 3s/step - loss: 3.1613 - val_loss: 0.8359
Epoch 5/50
1/1 [==============================] - 3s 3s/step - loss: 0.8466 - val_loss: 0.7215
Epoch 6/50
1/1 [==============================] - 3s 3s/step - loss: 0.7230 - val_loss: 0.6345
Epoch 7/50
1/1 [==============================] - 3s 3s/step - loss: 0.6454 - val_loss: 0.6129
Epoch 8/50
1/1 [==============================] - 3s 3s/step - loss: 0.6069 - val_loss: 0.5993
Epoch 9/50
1/1 [==============================] - 3s 3s/step - loss: 0.6211 - val_loss: 0.6247
Epoch 10/50
1/1 [==============================] - 3s 3s/step - loss: 0.6064 - val_loss: 0.5755
Epoch 11/50
1/1 [==============================

In [167]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index
source_word_index=x_tokenizer.word_index

In [168]:
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

dec_emb2= dec_emb_layer(decoder_inputs) 
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

decoder_outputs2 = decoder_dense(decoder_outputs2) 

decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [169]:
def decode_sequence(input_seq):
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    target_seq = np.zeros((1,1))
    
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
      output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

      sampled_token_index = np.argmax(output_tokens[0, -1, :])

      if(sampled_token_index != 0):
        sampled_token = reverse_target_word_index[sampled_token_index]
        decoded_sentence += ' '+sampled_token
      else:
        sampled_token = 'eostok'

      if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
          stop_condition = True

      target_seq = np.zeros((1,1))
      target_seq[0, 0] = sampled_token_index

      e_h, e_c = h, c

    return decoded_sentence

In [170]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [ ]:
sentence = input('Enter the sentence ')
sentenceList = []
index = 0
for i in range(0,65):
  sentenceList.append(0)
for word in sentence.split(" "):
  sentenceList[index] = source_word_index[word]
  index = index + 1
sentenceListAppend = []
sentenceListAppend.append(sentenceList)
print(sentenceListAppend)
print("Summary: ", decode_sequence(sentenceListAppend))